In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
%%time
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

CPU times: total: 0 ns
Wall time: 325 ms


200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)].reset_index(drop='index')
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
0,Зарубежные фильмы,21 Июн 23,Экзорцист Ватикана / The Pope's Exorcist (2023) WEB-DL 1080p от Scarabey | Red Head Sound,6.17 GB,1,124
1,Зарубежные фильмы,21 Июн 23,Один рейнджер / One Ranger (2023) WEB-DL 1080p от Scarabey | D | Red Head Sound,6.98 GB,2,511
2,Зарубежные фильмы,21 Июн 23,Нечто: Монстр из глубин / The Tank (2023) BDRip 1080p от New-Team | GoLTFilm,8.71 GB,160,167
3,Зарубежные фильмы,21 Июн 23,"Форсаж 10 / Fast X (2023) UHD WEB-DL-HEVC 2160p | 4K | HDR | Dolby Vision Profile 8 | P, A",28.60 GB,23,52
4,Зарубежные фильмы,21 Июн 23,Восстание зловещих мертвецов / Evil Dead Rise (2023) BDRip 720p от ExKinoRay | D,3.61 GB,127,45
5,Зарубежные фильмы,21 Июн 23,Удар / Punch (2022) WEB-DLRip от MegaPeer | Jaskier,1.46 GB,78,25
6,Зарубежные фильмы,20 Июн 23,"Уикенд с батей / About My Father (2023) WEB-DL-HEVC 2160p от Theseus | 4K | HDR | P | HDRezka Studio, Jaskier",16.04 GB,89,88
7,Зарубежные фильмы,20 Июн 23,Восстание зловещих мертвецов / Evil Dead Rise (2023) BDRip от MegaPeer | D,1.46 GB,59,14
8,Зарубежные сериалы,21 Июн 23,Секретное вторжение / Secret Invasion [01x01 из 06] (2023) WEB-DLRip 1080p | RG.Paravozik,1.85 GB,107,82
9,Зарубежные сериалы,21 Июн 23,"Извне / From [02x01-09 из 10] (2023) WEBRip 1080p | LostFilm, AlexFilm, HDRezka Studio",25.06 GB,9,139


In [8]:
id = [1, 2, 5, ]


my_df.loc[id, 'href']

1    http://rutor.info/torrent/930377/odin-rejndzher_one-ranger-2023-web-dl-1080p-ot-scarabey-d-red-head-sound
2      http://rutor.info/torrent/930375/nechto-monstr-iz-glubin_the-tank-2023-bdrip-1080p-ot-new-team-goltfilm
5                               http://rutor.info/torrent/930307/udar_punch-2022-web-dlrip-ot-megapeer-jaskier
Name: href, dtype: object